In [ ]:
# import pandas as pd
# import joblib
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.impute import SimpleImputer
# from sklearn.metrics import classification_report

# # ===============================

# # 1. Load merged dataset

# # ===============================

# df = pd.read_csv("Data/Nepal_building_with_Intensity.csv")

# # Check target column

# if "damage_grade" not in df.columns:
#     raise Exception("Dataset must contain 'damage_grade' column.")

# # Convert 0–5 → 0–10

# df["damage_score"] = (df["damage_grade"] / df["damage_grade"].max()) * 10

# # Categorize into low/medium/high

# df["damage_class"] = pd.cut(
# df["damage_score"],
# bins=[-0.1, 3.3, 6.6, 10],
# labels=["low", "medium", "high"]
# )


# print("Target distribution:\n", df["damage_class"].value_counts())

# # ===============================

# # 2. Define features

# # ===============================

# numeric_features = ["count_floors_pre_eq", "age_building", "plinth_area_sq_ft", "intensity"]
# categorical_features = ["foundation_type", "roof_type"]

# X = df[numeric_features + categorical_features]
# y = df["damage_class"]

# # ===============================

# # 3. Preprocessing pipeline with NaN handling

# # ===============================

# numeric_transformer = Pipeline([
# ("imputer", SimpleImputer(strategy="median")),  # fill NaNs with median
# ("scaler", StandardScaler())
# ])

# categorical_transformer = Pipeline([
# ("imputer", SimpleImputer(strategy="most_frequent")),  # fill NaNs with mode
# ("encoder", OneHotEncoder(handle_unknown="ignore"))
# ])

# preprocessor = ColumnTransformer(
# transformers=[
# ("num", numeric_transformer, numeric_features),
# ("cat", categorical_transformer, categorical_features)
# ]
# )

# # ===============================

# # 4. Train/test split

# # ===============================

# #drop the things with nan values


# # Drop rows with any NaN in features or target
# df = df.dropna(subset=numeric_features + categorical_features + ["damage_class"])

# X = df[numeric_features + categorical_features]
# y = df["damage_class"]

# # Now train_test_split will work
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )


# # ===============================

# # 5. Random Forest model pipeline

# # ===============================

# rf_pipeline = Pipeline([
# ("preprocessor", preprocessor),
# ("classifier", RandomForestClassifier(n_estimators=200, max_depth=12, random_state=42, class_weight="balanced"))
# ])

# # Train

# rf_pipeline.fit(X_train, y_train)

# # ===============================

# # 6. Evaluate

# # ===============================

# y_pred = rf_pipeline.predict(X_test)
# print("\nClassification Report:\n")
# print(classification_report(y_test, y_pred))

# # ===============================

# # 7. Save trained model

# # ===============================

# joblib.dump(rf_pipeline, "models/RandomForest.pkl")
# print("\nModel saved to models/RandomForest.pkl")


Target distribution:
 damage_class
high      459609
medium    223670
low        78815
Name: count, dtype: int64

Classification Report:

              precision    recall  f1-score   support

        high       0.83      0.69      0.75     91922
         low       0.43      0.74      0.54     15763
      medium       0.43      0.47      0.45     44734

    accuracy                           0.63    152419
   macro avg       0.56      0.63      0.58    152419
weighted avg       0.67      0.63      0.64    152419


Model saved to models/RandomForest.pkl


In [ ]:
# model = joblib.load("models/RandomForest.pkl")

In [ ]:
# samples = pd.DataFrame([
#     {   # Low damage
#         "count_floors_pre_eq": 1,
#         "age_building": 5,
#         "plinth_area_sq_ft": 50,
#         "intensity": 2.0,
#         "foundation_type": "Mud mortar-Stone/Brick",
#         "roof_type": "Bamboo/Timber-Light roof"
#     },
#     {   # Medium damage
#         "count_floors_pre_eq": 2,
#         "age_building": 25,
#         "plinth_area_sq_ft": 120,
#         "intensity": 4.5,
#         "foundation_type": "Mud mortar-Stone/Brick",
#         "roof_type": "Bamboo/Timber-Light roof"
#     },
#     {   # High damage
#         "count_floors_pre_eq": 4,
#         "age_building": 60,
#         "plinth_area_sq_ft": 200,
#         "intensity": 7.5,
#         "foundation_type": "Mud mortar-Stone/Brick",
#         "roof_type": "Bamboo/Timber-Light roof"
#     }
# ])

In [ ]:
# predicted_class = model.predict(samples)
# print("Predicted damage class: ", predicted_class)

# predicted_prob = model.predict_proba(samples)
# print("Predict probability: ", predicted_prob)

Predicted damage class:  ['medium' 'high' 'high']
Predict probability:  [[0.22161472 0.31202439 0.46636088]
 [0.52827983 0.10943107 0.3622891 ]
 [0.78210758 0.02521668 0.19267574]]


In [ ]:
# from sklearn.utils import resample

# # Separate classes
# df_low = df[df['damage_class'] == 'low']
# df_med = df[df['damage_class'] == 'medium']
# df_high = df[df['damage_class'] == 'high']

# # Upsample low and medium to match high
# df_low_upsampled = resample(df_low, replace=True, n_samples=len(df_high), random_state=42)
# df_med_upsampled = resample(df_med, replace=True, n_samples=len(df_high), random_state=42)

# # Combine
# df_balanced = pd.concat([df_low_upsampled, df_med_upsampled, df_high])


In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier(
#     n_estimators=200,
#     max_depth=12,
#     random_state=42,
#     class_weight="balanced"
# )


In [4]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.utils import resample

# ===============================

# 1. Load dataset

# ===============================

df = pd.read_csv("Data/Nepal_building_with_Intensity.csv")

if "damage_grade" not in df.columns:
    raise Exception("Dataset must contain 'damage_grade' column.")

# Convert 0-5 → 0-10

df["damage_score"] = (df["damage_grade"] / df["damage_grade"].max()) * 10

# Categorize into low/medium/high

df["damage_class"] = pd.cut(
df["damage_score"],
bins=[-0.1, 3.3, 6.6, 10],
labels=["low", "medium", "high"]
)

print("Original class distribution:\n", df["damage_class"].value_counts())

# ===============================

# 2. Balance classes by upsampling

# ===============================

df_low = df[df['damage_class'] == 'low']
df_med = df[df['damage_class'] == 'medium']
df_high = df[df['damage_class'] == 'high']

df_low_up = resample(df_low, replace=True, n_samples=len(df_low), random_state=42)
df_med_up = resample(df_med, replace=True, n_samples=len(df_low), random_state=42)

df_balanced = pd.concat([df_high, df_low_up, df_med_up])
# print(df_balanced.shape)
# print("\nBalanced class distribution:\n", df_balanced["damage_class"].value_counts())
# df_balanced.drop_duplicates(inplace=True)

# print(df_balanced.shape)
# ===============================

# 3. Define features

# ===============================

numeric_features = ["count_floors_pre_eq", "age_building", "plinth_area_sq_ft", "intensity"]
categorical_features = ["foundation_type", "roof_type"]

X = df_balanced[numeric_features + categorical_features]
y = df_balanced["damage_class"]
X.shape
y.shape
# ===============================

# 4. Preprocessing pipeline with NaN handling

# ===============================

numeric_transformer = Pipeline([
("imputer", SimpleImputer(strategy="median")),
("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
("imputer", SimpleImputer(strategy="most_frequent")),
("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
("num", numeric_transformer, numeric_features),
("cat", categorical_transformer, categorical_features)
])

# ===============================

# 5. Train/test split

# ===============================

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42, stratify=y
)

# ===============================

# 6. Random Forest pipeline

# ===============================

rf_pipeline = Pipeline([
("preprocessor", preprocessor),
("classifier", RandomForestClassifier(
n_estimators=200,
max_depth=12,
random_state=42,
class_weight="balanced"
))
])

# Train

rf_pipeline.fit(X_train, y_train)

# ===============================

# 7. Evaluate

# ===============================

y_pred = rf_pipeline.predict(X_test)
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# ===============================

# 8. Save model

# ===============================

joblib.dump(rf_pipeline, "models/RandomForest_balanced.pkl")
print("\nModel saved to models/RandomForest_balanced.pkl")

# ===============================

# 9. Generate synthetic test data

# ===============================

np.random.seed(42)
n_samples = 15

test_df = pd.DataFrame({
"count_floors_pre_eq": np.random.randint(1, 6, n_samples),
"age_building": np.random.randint(1, 80, n_samples),
"plinth_area_sq_ft": np.random.randint(30, 250, n_samples),
"intensity": np.random.uniform(2, 8, n_samples),
"foundation_type": np.random.choice(
["Mud mortar-Stone/Brick","RCC","Brick-Concrete","Stone"], n_samples
),
"roof_type": np.random.choice(
["Bamboo/Timber-Light roof","RCC","Tile","Metal"], n_samples
)
})

print("\nSynthetic test data:\n", test_df)

# ===============================

# 10. Predict on synthetic test data

# ===============================

preds = rf_pipeline.predict(test_df)
print("\nPredictions:\n", preds)


Original class distribution:
 damage_class
high      459609
medium    223670
low        78815
Name: count, dtype: int64

Classification Report:

              precision    recall  f1-score   support

        high       0.93      0.69      0.79     91922
         low       0.60      0.76      0.67     15763
      medium       0.22      0.47      0.30     15763

    accuracy                           0.67    123448
   macro avg       0.58      0.64      0.59    123448
weighted avg       0.79      0.67      0.71    123448


Model saved to models/RandomForest_balanced.pkl

Synthetic test data:
     count_floors_pre_eq  age_building  plinth_area_sq_ft  intensity  \
0                     4            30                219   7.693313   
1                     5            38                219   7.793792   
2                     3             2                204   6.850384   
3                     5            64                219   3.827683   
4                     5            60          

In [ ]:
samples = pd.DataFrame([
    {   # High damage
        "count_floors_pre_eq": 2,
        "age_building": 20,
        "plinth_area_sq_ft": 1000,
        "intensity": 7.5
    }
])

In [46]:
preds = rf_pipeline.predict(samples)
print("\nPredictions:\n", preds)


Predictions:
 ['medium' 'high' 'high']


In [49]:
df.value_counts

<bound method DataFrame.value_counts of          building_id  district_id  vdcmun_id  ward_id  count_floors_pre_eq  \
0       1.201010e+11           12       1207   120703                    1   
1       1.201010e+11           12       1207   120703                    1   
2       1.201010e+11           12       1207   120703                    1   
3       1.201010e+11           12       1207   120703                    1   
4       1.201010e+11           12       1207   120703                    1   
...              ...          ...        ...      ...                  ...   
762101  3.667090e+11           36       3603   360302                    2   
762102  3.667090e+11           36       3603   360302                    2   
762103  3.667090e+11           36       3603   360302                    2   
762104  3.667090e+11           36       3603   360302                    2   
762105  3.667090e+11           36       3603   360302                    2   

        count_floors_po